<a href="https://colab.research.google.com/github/shepherdjerred/time-off/blob/main/time_off.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import datetime
from dateutil.relativedelta import relativedelta
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.dates as mdates

In [8]:
paid_time_off_hours_first_paycheck_or_january = 10
paid_time_off_hours_months_two_through_five = 8
paid_time_off_hours_month_six = 6
vacation_hours_per_month = 10
sick_time_hours_per_month = 2

# Cannot accumulate more than this many vacation hours
max_vacation_time = 160

# Hours over this amount will be compensated at my hourly rate up to the amount defined below
paid_time_off_max_yearly_rollover = 48

# Hours over this amount will be wasted at the end of the year
paid_time_off_and_sick_time_max_compensated_yearly_rollover = 72

In [9]:
class Change:
    def __init__(self, date, reason, sick_time_change = 0, paid_time_off_change = 0, vacation_time_change = 0):
        self.date = date
        self.reason = reason
        self.sick_time_change = sick_time_change
        self.paid_time_off_change = paid_time_off_change
        self.vacation_time_change = vacation_time_change
    
    def to_dict(self):
        return {
            'date': self.date,
            'reason': self.reason,
            'sick_time_change': self.sick_time_change,
            'paid_time_off_change': self.paid_time_off_change,
            'vacation_time_change': self.vacation_time_change
        }

In [10]:
changes = [
    # 2019
    Change(datetime.datetime(2019,  7, 21), 'Mystery Adjustment ???',   15,  14, 0),
    Change(datetime.datetime(2019,  8,  1), 'July Paycheck',             2,  10, 6.67),
    Change(datetime.datetime(2019,  9,  1), 'August Paycheck',           2,  8,  6.67),
    Change(datetime.datetime(2019, 10,  1), 'September Paycheck',        2,  8,  6.67),
    Change(datetime.datetime(2019, 10, 17), 'Homecoming',                0, -8,  0),
    Change(datetime.datetime(2019, 10, 18), 'Homecoming',                0, -8,  0),
    Change(datetime.datetime(2019, 10, 21), 'Homecoming',                0, -8,  0),
    Change(datetime.datetime(2019, 11,  1), 'October Paycheck',          2,  8,  6.67),
    Change(datetime.datetime(2019, 11, 26), 'Back hurt',                -8,  0,  0),
    Change(datetime.datetime(2019, 11, 29), 'Black Friday',              0, -8,  0),
    Change(datetime.datetime(2019, 12,  1), 'November Paycheck',         2,  8,  6.67),
    Change(datetime.datetime(2019, 12, 31), 'Skiing',                    0,  0, -8),
    # 2020
    Change(datetime.datetime(2020,  1,  1), 'December Paycheck',         2, 10,  6.67),
    Change(datetime.datetime(2020,  1,  6), 'Skiing/Travelling',         0,  0, -8),
    Change(datetime.datetime(2020,  2,  1), 'January Paycheck',          2,  8,  6.67),
    Change(datetime.datetime(2020,  2, 17), 'Skiing (not really)',       0,  0, -8),
    Change(datetime.datetime(2020,  2, 20), 'Sore Throat',              -8,  0,  0),
    Change(datetime.datetime(2020,  2, 21), 'Sore Throat',              -8,  0,  0),
    Change(datetime.datetime(2020,  3,  1), 'February Paycheck',         2,  8,  6.67),
    Change(datetime.datetime(2020,  3, 19), 'Mystery Adjustment ???',    0, -8,  0),
    Change(datetime.datetime(2020,  4,  1), 'March Paycheck',            2,  8,  6.67),
    Change(datetime.datetime(2020,  5,  1), 'April Paycheck',            2,  8,  6.67),
    Change(datetime.datetime(2020,  6,  1), 'May Paycheck',              2,  6,  6.67),
    Change(datetime.datetime(2020,  7,  1), 'June Paycheck',             2,  0,  6.67),
    Change(datetime.datetime(2020,  7,  2), 'Missed "Take a Break" day', 0,  8,  0),    # Unofficial
    Change(datetime.datetime(2020,  8,  1), 'July Paycheck',             2,  0,  10),
    Change(datetime.datetime(2020,  8, 21), 'Headache',                 -4,  0,  0),
    Change(datetime.datetime(2020,  8, 27), 'Factorio Day',              0, -8,  0),
    Change(datetime.datetime(2020,  8, 28), 'Factorio Day Pt 2',         0, -8,  0),
    Change(datetime.datetime(2020,  9,  1), 'August Paycheck',           2,  0,  10),
    Change(datetime.datetime(2020, 10,  1), 'September Paycheck',        2,  0,  10),
    Change(datetime.datetime(2020, 11,  1), 'October Paycheck',          2,  0,  10),
    Change(datetime.datetime(2020, 12,  1), 'November Paycheck',         2,  0,  10),
    Change(datetime.datetime(2020, 12, 11), 'Day off',                    0,  0, -8),
    Change(datetime.datetime(2020, 12, 15), 'Skiing',                    0,  0, -8),
    Change(datetime.datetime(2020, 12, 21), 'Day off',                    0,  0, -8),
    Change(datetime.datetime(2020, 12, 24), 'Christmas Eve off',          0,  0, -8),
    Change(datetime.datetime(2020, 12, 28), 'Delaney Visiting',          0,  0, -8),
    Change(datetime.datetime(2020, 12, 29), 'Delaney Visiting',          0,  0, -8),
    # 2021
    Change(datetime.datetime(2021,  1,  1), 'December Paycheck',         2,  0,  10),
    Change(datetime.datetime(2021,  1, 21), 'Day off',                    0,  0, -8),
    Change(datetime.datetime(2021,  2,  1), 'January Paycheck',          2,  8,  10),
    Change(datetime.datetime(2021,  2, 15), 'Day off',                    0, -8,  0),
    Change(datetime.datetime(2021,  3,  1), 'February Paycheck',         2,  8,  10),
    Change(datetime.datetime(2021,  3, 1,), 'Mystery Adjustment ???',    1,  2,  8),
    Change(datetime.datetime(2021,  4,  1), 'March Paycheck',            2,  8,  10),
    Change(datetime.datetime(2021,  4, 7),  'California',                0, -8,  0),
    Change(datetime.datetime(2021,  4, 8),  'California',                0, -8,  0),
    Change(datetime.datetime(2021,  4, 9),  'California',                0, -8,  0),
    Change(datetime.datetime(2021,  4, 12),  'Relax',                    0, -8,  0),
    Change(datetime.datetime(2021,  4, 13),  'Relax',                    0, -8,  0),
    Change(datetime.datetime(2021,  4, 14),  'Relax',                    0, -8,  0),
    Change(datetime.datetime(2021,  4, 15),  'Relax',                    0, -8,  0),
    Change(datetime.datetime(2021,  4, 16),  'Relax',                    0, -8,  0),
    Change(datetime.datetime(2021,  5,  1), 'April Paycheck',            2,  8,  10),
    Change(datetime.datetime(2021,  5,  24), '"Sick from Vaccine"',     -8,  0,  0),
    Change(datetime.datetime(2021,  6,  1), 'May Paycheck',              2,  8,  10),
    Change(datetime.datetime(2021,  6,  8), 'Ashton & Molly',            0,  -8,  0),
    Change(datetime.datetime(2021,  6,  9), 'Ashton & Molly',            0,  -8,  0),
    Change(datetime.datetime(2021,  6,  10), 'Ashton & Molly',           0,  -8,  0),
    Change(datetime.datetime(2021,  6,  11), 'Ashton & Molly',           0,   0, -8),
    # Change(datetime.datetime(2021,  7,  1), 'June Paycheck',             2,  8,  10),
]

changes_dataframe = pd.DataFrame.from_records([change.to_dict() for change in changes])

In [11]:
today = datetime.datetime.today()
simulation_length_in_months = 6

simulated_changes = []

for month in range(1, simulation_length_in_months + 1):
    projection_date = today + relativedelta(months=month, days=0, hour=0, minute=0, second=0, microsecond=0)
    projection_month = projection_date.month

    projection_sick_time_change = sick_time_hours_per_month
    projection_vacation_change = vacation_hours_per_month

    if projection_month == 1:
        projection_paid_time_off_change = paid_time_off_hours_first_paycheck_or_january
    elif projection_month < 6:
        projection_paid_time_off_change = paid_time_off_hours_months_two_through_five
    elif projection_month == 6:
        projection_paid_time_off_change = paid_time_off_hours_month_six
    else:
        projection_paid_time_off_change = 0

    month_change = Change(projection_date, f'FUTURE PAYCHECK: {projection_date.strftime("%B %Y")}', projection_sick_time_change, projection_paid_time_off_change, projection_vacation_change)
    simulated_changes.append(month_change)


projection_changes_dataframe = pd.DataFrame.from_records([change.to_dict() for change in simulated_changes])

In [12]:
projection_plus_existing_changes = pd.concat([changes_dataframe, projection_changes_dataframe])

projection_plus_existing_changes = projection_plus_existing_changes.assign(sick_time_hours=projection_plus_existing_changes['sick_time_change'].cumsum())
projection_plus_existing_changes = projection_plus_existing_changes.assign(paid_time_off_hours=projection_plus_existing_changes['paid_time_off_change'].cumsum())
projection_plus_existing_changes = projection_plus_existing_changes.assign(vacation_time_hours=projection_plus_existing_changes['vacation_time_change'].cumsum())
projection_plus_existing_changes = projection_plus_existing_changes

projection_plus_existing_changes['total_sick_time_days'] = projection_plus_existing_changes['sick_time_hours'].apply(lambda value : math.floor(value / 8))
projection_plus_existing_changes['total_paid_time_off_days'] = projection_plus_existing_changes['paid_time_off_hours'].apply(lambda value : math.floor(value / 8))
projection_plus_existing_changes['total_vacation_time_days'] = projection_plus_existing_changes['vacation_time_hours'].apply(lambda value : math.floor(value / 8))
projection_plus_existing_changes['total_leisure_days'] = round((projection_plus_existing_changes['paid_time_off_hours'] + projection_plus_existing_changes['vacation_time_hours']) / 8, 0)

display = projection_plus_existing_changes
display = display.drop(['sick_time_change', 'paid_time_off_change', 'vacation_time_change'], 1)
# display = display.drop(['total_sick_time_days', 'total_paid_time_off_days', 'total_vacation_time_days', 'total_leisure_days'], 1)
display = display.drop(['sick_time_hours', 'paid_time_off_hours', 'vacation_time_hours'], 1)

display[-13:]

,date,reason,total_sick_time_days,total_paid_time_off_days,total_vacation_time_days,total_leisure_days
54,2021-05-01,April Paycheck,4,2,13,16.0
55,2021-05-24,"""Sick from Vaccine""",3,2,13,16.0
56,2021-06-01,May Paycheck,3,3,14,18.0
57,2021-06-08,Ashton & Molly,3,2,14,17.0
58,2021-06-09,Ashton & Molly,3,1,14,16.0
59,2021-06-10,Ashton & Molly,3,0,14,15.0
60,2021-06-11,Ashton & Molly,3,0,13,14.0
0,2021-06-29,FUTURE PAYCHECK: June 2021,3,1,15,16.0
1,2021-07-29,FUTURE PAYCHECK: July 2021,3,1,16,17.0
2,2021-08-29,FUTURE PAYCHECK: August 2021,4,1,17,19.0
